<a href="https://colab.research.google.com/github/AntonioFialhoSN/Entrega_da_P2_RedesNeurais/blob/main/Callback_PyTorch_BaseCSVWriter.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
pip install torch

In [ ]:
# Importar bibliotecas necessárias
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split

In [ ]:
class BaseCSVWriter:
    """
    Callback para escrever métricas de treinamento em um arquivo CSV.

    Args:
        filename (str): Nome do arquivo CSV de saída.
        metrics (list): Lista de métricas a serem registradas.
    """
    def __init__(self, filename="training_log.csv", metrics=['epoch', 'loss', 'accuracy']):
        self.filename = filename
        self.metrics = metrics
        # Cria o arquivo CSV com cabeçalho
        pd.DataFrame(columns=self.metrics).to_csv(self.filename, index=False)

    def __call__(self, epoch, logs):
        """
        Método chamado a cada época para registrar as métricas.

        Args:
            epoch (int): Número da época atual.
            logs (dict): Dicionário com as métricas de treinamento/validação.
        """
        # Prepara os dados para esta época
        log_data = {metric: logs.get(metric, None) for metric in self.metrics}
        log_data['epoch'] = epoch

        # Converte para DataFrame e salva no CSV (append)
        pd.DataFrame([log_data]).to_csv(
            self.filename,
            mode='a',
            header=False,
            index=False
        )

        print(f"Métricas salvas em {self.filename} para a época {epoch}")

In [ ]:
# Gerar dados sintéticos
np.random.seed(42)
X = np.random.rand(1000, 10)  # 1000 amostras, 10 features
y = (np.sum(X, axis=1) > 5).astype(int)  # Classificação binária baseada na soma

# Dividir em treino e teste
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Converter para tensores PyTorch
X_train = torch.FloatTensor(X_train)
X_test = torch.FloatTensor(X_test)
y_train = torch.LongTensor(y_train)
y_test = torch.LongTensor(y_test)

# Criar DataLoaders
train_data = TensorDataset(X_train, y_train)
test_data = TensorDataset(X_test, y_test)
train_loader = DataLoader(train_data, batch_size=32, shuffle=True)
test_loader = DataLoader(test_data, batch_size=32)

In [ ]:
class SimpleNN(nn.Module):
    """Rede neural simples para classificação binária."""

    def __init__(self, input_size):
        super(SimpleNN, self).__init__()
        # Define as camadas da rede:
        self.fc1 = nn.Linear(input_size, 16)  # 1ª camada densa: input_size -> 16 neurônios
        self.fc2 = nn.Linear(16, 8)           # 2ª camada densa: 16 -> 8 neurônios
        self.fc3 = nn.Linear(8, 2)            # 3ª camada densa: 8 -> 2 neurônios (saída binária)

        # Funções de ativação:
        self.relu = nn.ReLU()                 # ReLU para não-linearidade
        self.softmax = nn.Softmax(dim=1)      # Softmax para probabilidades de classe

    def forward(self, x):
        # Fluxo de dados (forward pass):
        x = self.relu(self.fc1(x))  # Aplica fc1 seguido de ReLU
        x = self.relu(self.fc2(x))  # Aplica fc2 seguido de ReLU
        x = self.fc3(x)             # Aplica fc3 (sem ativação antes do softmax)
        return self.softmax(x)      # Retorna probabilidades via softmax

In [ ]:
def train_model(model, train_loader, test_loader, epochs=10, lr=0.001):
    """Função para treinar o modelo com registro em CSV."""
    # Inicializar otimizador e função de perda
    optimizer = optim.Adam(model.parameters(), lr=lr)
    criterion = nn.CrossEntropyLoss()

    # Inicializar a callback CSVWriter
    csv_writer = BaseCSVWriter("training_log.csv")

    for epoch in range(epochs):
        model.train()
        train_loss = 0.0
        correct = 0
        total = 0

        # Loop de treinamento
        for inputs, labels in train_loader:
            optimizer.zero_grad()
            outputs = model(inputs)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()

            train_loss += loss.item()
            _, predicted = torch.max(outputs.data, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()

        # Calcular métricas
        train_accuracy = correct / total
        avg_train_loss = train_loss / len(train_loader)

        # Avaliar no conjunto de teste
        test_loss, test_accuracy = evaluate_model(model, test_loader, criterion)

        # Preparar logs para a callback
        logs = {
            'loss': avg_train_loss,
            'accuracy': train_accuracy,
            'val_loss': test_loss,
            'val_accuracy': test_accuracy
        }

        # Chamar a callback para registrar as métricas
        csv_writer(epoch, logs)

        print(f"Época {epoch+1}/{epochs}")
        print(f"Train Loss: {avg_train_loss:.4f}, Accuracy: {train_accuracy:.4f}")
        print(f"Test Loss: {test_loss:.4f}, Accuracy: {test_accuracy:.4f}")
        print("---------------------")

def evaluate_model(model, loader, criterion):
    """Avaliar o modelo em um conjunto de dados."""
    model.eval()
    loss = 0.0
    correct = 0
    total = 0

    with torch.no_grad():
        for inputs, labels in loader:
            outputs = model(inputs)
            loss += criterion(outputs, labels).item()
            _, predicted = torch.max(outputs.data, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()

    return loss / len(loader), correct / total

In [ ]:
# Inicializar o modelo
model = SimpleNN(input_size=10)

# Treinar o modelo
train_model(model, train_loader, test_loader, epochs=5)

# Mostrar o conteúdo do arquivo CSV gerado
print("\nConteúdo do arquivo CSV:")
print(pd.read_csv("training_log.csv"))

Métricas salvas em training_log.csv para a época 0
Época 1/5
Train Loss: 0.6926, Accuracy: 0.4750
Test Loss: 0.6983, Accuracy: 0.4200
---------------------
Métricas salvas em training_log.csv para a época 1
Época 2/5
Train Loss: 0.6872, Accuracy: 0.4775
Test Loss: 0.6919, Accuracy: 0.4150
---------------------
Métricas salvas em training_log.csv para a época 2
Época 3/5
Train Loss: 0.6808, Accuracy: 0.5188
Test Loss: 0.6854, Accuracy: 0.4800
---------------------
Métricas salvas em training_log.csv para a época 3
Época 4/5
Train Loss: 0.6734, Accuracy: 0.5537
Test Loss: 0.6761, Accuracy: 0.5950
---------------------
Métricas salvas em training_log.csv para a época 4
Época 5/5
Train Loss: 0.6634, Accuracy: 0.6288
Test Loss: 0.6672, Accuracy: 0.6000
---------------------

Conteúdo do arquivo CSV:
   epoch      loss  accuracy
0      0  0.692566   0.47500
1      1  0.687156   0.47750
2      2  0.680757   0.51875
3      3  0.673417   0.55375
4      4  0.663388   0.62875
